https://github.com/linanqiu/word2vec-sentiments/blob/master/word2vec-sentiment.ipynb
    
https://github.com/bgshin/sentiment

In [1]:
#basic imports 

from itertools import chain

from gensim.models import Word2Vec, KeyedVectors

import numpy as np

In [2]:
pwd

u'/home/root1/anuj_work/Github_repos/NLP_by_Example/Ch9'

In [3]:
# load Google's word2vec model

#path_to_model = '../../jupyters/vocab_coverage/word2vec_web/GoogleNews-vectors-negative300.bin'
path_to_model = './../../../GoogleNews-vectors-negative300.bin'
#w2v_model = Word2Vec.load_word2vec_format(path_to_model, binary=True)
w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print('done loading Word2Vec')

done loading Word2Vec


In [4]:
# get word2vec's vocab
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]

print len(word2vec_vocab)

3000000


In [11]:
def import_tag(datasets=None):
    ''' Imports the datasets into one of two dictionaries

        Dicts:
            train & test

        Keys:
            values >= 4  are "Positive" in both Dictionaries

        '''
    if datasets is not None:
        train_dict = {}
        test_dict = {}
        
        #train_X = []
        #train_Y = [] 
        train_Y = [] 
        test_Y = []
        
        print datasets.items()

        for k, v in datasets.items(): #gives a list of tuples to iterate
            with open(k) as fpath:
                data = fpath.readlines()
                
                print len(data)
                
            for val, each_line in enumerate(data):
                
                if v.endswith("NEG") and v.startswith("TRAIN"):
                    try:
                        train_dict[val] = each_line
                        #train_Y[val] = 0
                        train_Y.append(0)
                    except IndexError:
                        print val
                elif v.endswith("POS") and v.startswith("TRAIN"):
                    train_dict[val + 12500] = each_line
                    train_Y.append(1)
                
                elif v.endswith("NEG") and v.startswith("TEST"):
                    try:
                        test_dict[val] = each_line
                        #test_Y[val] = 0
                        test_Y.append(0)
                    except IndexError:
                        print val
                else:
                    test_dict[val + 12500] = each_line
                    #test_Y[val] = 1
                    test_Y.append(1)
                        
                        
        return train_dict, test_dict,train_Y, test_Y
    else:
        print('Data not found...')

In [12]:
#cd ./../../../dataset/


In [13]:
#cd /home/root1/anuj_work/Github_repos/NLP_by_Example/Ch10

In [14]:
data_locations = {'./../../../dataset/test-neg.txt': 'TEST_NEG',
                  './../../../dataset/test-pos.txt': 'TEST_POS',
                  './../../../dataset/train-neg.txt': 'TRAIN_NEG',
                  './../../../dataset/train-pos.txt': 'TRAIN_POS'}

In [15]:
#pwd

In [16]:
#s ./../../../dataset/test-neg.txt

data_locations = {'./../../jupyters/Data/test-neg.txt': 'TEST_NEG',
                  './../../jupyters/Data/test-pos.txt': 'TEST_POS',
                  './../../jupyters/Data/train-neg.txt': 'TRAIN_NEG',
                  './../../jupyters/Data/train-pos.txt': 'TRAIN_POS'}

In [17]:
train_dict, test_dict, train_Y,test_Y = import_tag(data_locations)

[('./../../../dataset/test-neg.txt', 'TEST_NEG'), ('./../../../dataset/train-neg.txt', 'TRAIN_NEG'), ('./../../../dataset/test-pos.txt', 'TEST_POS'), ('./../../../dataset/train-pos.txt', 'TRAIN_POS')]
12500
12500
12500
12500


In [18]:
type(train_dict)

dict

In [19]:
train_dict[0]

'story of a man who has unnatural feelings for a pig starts out with a opening scene that is a terrific example of absurd comedy a formal orchestra audience is turned into an insane violent mob by the crazy chantings of it s singers unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting even those from the era should be turned off the cryptic dialogue would make shakespeare seem easy to a third grader on a technical level it s better than you might think with some good cinematography by future great vilmos zsigmond future stars sally kirkland and frederic forrest can be seen briefly \n'

In [20]:
def tokenizer(text):
    ''' Simple Parser converting each document to lower-case, then
        removing the breaks for new lines and finally splitting on the
        whitespace
    '''
    text = [document.lower().replace('\n', '').split() for document in text]
    return text

In [21]:
# combine training and test reviews (50K) and tokenize each one of them

print('Loading Data...')
combined = train_dict.values() + test_dict.values()
print('Tokenising...')
combined = tokenizer(combined)

Loading Data...
Tokenising...


In [22]:
print type(combined), len(combined)
print combined[0]

<type 'list'> 50000
['story', 'of', 'a', 'man', 'who', 'has', 'unnatural', 'feelings', 'for', 'a', 'pig', 'starts', 'out', 'with', 'a', 'opening', 'scene', 'that', 'is', 'a', 'terrific', 'example', 'of', 'absurd', 'comedy', 'a', 'formal', 'orchestra', 'audience', 'is', 'turned', 'into', 'an', 'insane', 'violent', 'mob', 'by', 'the', 'crazy', 'chantings', 'of', 'it', 's', 'singers', 'unfortunately', 'it', 'stays', 'absurd', 'the', 'whole', 'time', 'with', 'no', 'general', 'narrative', 'eventually', 'making', 'it', 'just', 'too', 'off', 'putting', 'even', 'those', 'from', 'the', 'era', 'should', 'be', 'turned', 'off', 'the', 'cryptic', 'dialogue', 'would', 'make', 'shakespeare', 'seem', 'easy', 'to', 'a', 'third', 'grader', 'on', 'a', 'technical', 'level', 'it', 's', 'better', 'than', 'you', 'might', 'think', 'with', 'some', 'good', 'cinematography', 'by', 'future', 'great', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'and', 'frederic', 'forrest', 'can', 'be', 'seen', '

In [23]:
#  get the vocab size of w2v_model
len(w2v_model.vocab.keys())

# this is one way to create vocab set from our dataset 
flat_list = chain(*combined)
dataset_vocab = set(flat_list)
len(dataset_vocab)
dataset_vocab_lower = [key.lower() for key in dataset_vocab]

In [24]:
# This does 2 things - 
# 1) Create a small dictionary of word to vectors for words which are only present in our dataset
# 2) Count the number of words in our corpus not present in Google's word to vec

DIMENSION = 300
zero_vector = np.zeros(DIMENSION)

our_dict = {}
missing_count = 0

for key in dataset_vocab_lower:
    if key in w2v_model:
        our_dict[key] = w2v_model[key]
    else:
        missing_count += 1
        #print("missing key %s" %(key)) 
        our_dict[key] = zero_vector

print len(dataset_vocab_lower)
print missing_count

100673
41929


In [25]:
# fetch word vector for each word. Add all the word vectors in a column sum fashion to get a 
# single vector of (DIMENSION,)

def getWordVecs(words, w2v_dict):
    vecs = []
    for word in words:
        word = word.replace('\n', '')
        try:
            vecs.append(w2v_dict[word].reshape((1,300)))
        except KeyError:
            continue
    vecs = np.concatenate(vecs)
    vecs = np.array(vecs, dtype='float')
    final_vec = np.sum(vecs, axis=0)
    return final_vec

In [26]:

def process_data(combined, w2v_dict, train_X, train_Y, test_X, test_Y):
    
    for i, data in enumerate(combined):
        try:
            vector_rep = getWordVecs(data, w2v_dict)
        except IndexError:
            print i
        
        if i < 25000:
            train_X.append(vector_rep)
        else:
            test_X.append(vector_rep)
            
        train_X_ = np.array(train_X, dtype='float')
        train_X_.shape

        train_Y_ = np.array(train_Y, dtype='int')
        train_Y_.shape

        test_X_ = np.array(test_X, dtype='float')
        test_X_.shape

        test_Y_ = np.array(test_Y, dtype='int')
        test_Y_.shape

    
    return train_X_, train_Y_, test_X_, test_Y_
    

In [27]:
print len(combined)

50000


In [28]:
train_X = []
test_X = []
train_X, train_Y, test_X, test_Y = process_data(combined, our_dict, train_X, train_Y, test_X, test_Y)

In [29]:
print type(train_X)
print type(test_X)

print type(train_Y)
print type(test_Y)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [ ]:
# we now have the data loaded, processed and featureized (using word vectors)

# we are now ready top train our classifier

In [ ]:
# visualize if the features are good enough to give a clean seperation

# we do this using t-SNE

In [30]:
# Data processing done, lets run a classfier

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
classifier = LogisticRegression()
classifier.fit(train_X, train_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
classifier.score(test_X, test_Y)

0.85888

Not bad. With little efforts we got 85% accuracy. Thats a great basline to have